In [6]:
#!fsharp

open System.Linq
open System.Text.Json

/// <summary>
/// Wraps <see cref="JsonException" /> property
/// in <see cref="Error" />.
/// </summary>
/// <param name="elementName">The <see cref="JsonElement" /> name.</param>
let resultError (elementName: string) =
    Error(JsonException $"the expected `{elementName}` element is not here.")

/// <summary>
/// Tries to return the <see cref="JsonElement" /> property
/// of the specified <see cref="JsonElement" /> object.
/// </summary>
/// <param name="elementName">The <see cref="JsonElement" /> name.</param>
/// <param name="element">The <see cref="JsonElement" />.</param>
let tryGetProperty (elementName: string) (element: JsonElement) =
    match element.TryGetProperty elementName with
    | false, _ -> resultError elementName
    | true, el -> Ok el

/// <summary>
/// Converts the <see cref="JsonElement" />
/// to its property name or returns <see cref="None" />.
/// </summary>
/// <param name="element">The <see cref="JsonElement" />.</param>
let toPropertyName (element: JsonElement) =
    if element.ValueKind <> JsonValueKind.Object then None
    else
        try
            Some (element.EnumerateObject().First().Name)
        with | _ -> None


In [7]:
#!fsharp

let jDoc = JsonDocument.Parse(@"
    {
        ""top"": {
            ""one"": ""this is first"",
            ""two"": ""this is second"",
            ""three"": ""this is third""
        }
    }
")

In [8]:
#!fsharp

jDoc.RootElement |> toPropertyName

Value,top


In [9]:
#!fsharp

jDoc.RootElement |> tryGetProperty "one"

Microsoft.FSharp.Core.FSharpResult`2[System.Text.Json.JsonElement,System.Text.Json.JsonException] ResultValue ValueKind Undefined ErrorValue System.Text.Json.JsonException: the expected `one` element is not here. LineNumber <null> BytePositionInLine <null> Path <null> Message the expected `one` element is not here. TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233088 StackTrace <null>

In [11]:
#!fsharp

jDoc.RootElement |> tryGetProperty "top"

Microsoft.FSharp.Core.FSharpResult`2[System.Text.Json.JsonElement,System.Text.Json.JsonException] ResultValue {
 "one": "this is first",
 "two": "this is second",
 "three": "this is third"
 } one "this is first" two "this is second" three "this is third" ErrorValue <null>